In [137]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from geoalchemy2 import Geometry, WKTElement
from shapely.geometry import Point
from sqlalchemy.types import Integer, String, Float
import requests
from io import StringIO
import configparser
config=configparser.ConfigParser()


In [138]:
config.read('config.cfg')
host=config['credential']['host']
database=config['credential']['database']
user=config['credential']['user']
password=config['credential']['password']
port=config['credential']['port']

In [139]:
def download_and_clean_airport_df():
    seession=requests.get('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat')
    s=str(seession.content,'utf-8')
    data=StringIO(s)
    # Load the airports data and clean it
    df = pd.read_csv(data, header=None, names=[
    "airport_id", "name", "city", "country", "iata_code", "icao_code", 
    "latitude", "longitude", "tltitude", "timezone", 
    "dst", "tz_database", "type", "source"
])
    print(df.shape[0])
    df['location'] = df.apply(
        lambda row: WKTElement(
            Point(row.longitude, row.latitude).wkt,
            srid=4326
        ),
        axis=1
    )
    df['timezone']=df['timezone'].str.replace('\\N', '')
    df=df[df['timezone']!='']
    df.iata_code= df.iata_code.str.strip().str.upper()
    df.icao_code= df.icao_code.str.strip().str.upper()
    df.name= df.name.str.strip().str.title()   
    df.city= df.city.str.strip().str.title() 
    df.country= df.country.str.strip().str.title() 
    df.drop_duplicates(['airport_id'],inplace=True)
    df.reset_index(drop=True,inplace=True)
    return df

In [140]:
def download_and_clean_airline_df():
    seession=requests.get('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat')
    s=str(seession.content,'utf-8')
    data=StringIO(s)
    # Load the airlines data and clean it
    df = pd.read_csv(data, header=None, names=[
    "airline_id", "name", "alias", "iata_code", "icao_code", 
    "callsign", "country", "active"
        ])
    print(df.shape[0])
    df=df[df.iata_code.str.len()==2]
    df.name= df.name.str.strip().str.title()
    df.country= df.country.str.strip().str.title() 
    df.iata_code= df.iata_code.str.strip().str.upper()
    df.icao_code= df.icao_code.str.strip().str.upper()
    df.drop_duplicates(['airline_id'],inplace=True)
   
    df.reset_index(drop=True,inplace=True)
    return df

In [141]:
def download_and_clean_routes_df():
    seession=requests.get('https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat')
    s=str(seession.content,'utf-8')
    data=StringIO(s)
    # Load the routes data and clean it
    df = pd.read_csv(data, header=None, names=[
        "airline", "airline_id", "source_airport", "source_airport_id", 
        "destination_airport", "destination_airport_id", "codeshare", 
        "stops", "equipment"
    ])
    print(df.shape[0])
    df=df[df['source_airport_id']!='\\N']
    df=df[df['destination_airport_id']!='\\N']
    df=df[df['airline_id']!='\\N'].reset_index(drop=True)
    df.airline=df.airline.str.strip().str.upper()
    df.source_airport=df.source_airport.str.strip().str.upper()
    df.destination_airport=df.destination_airport.str.strip().str.upper()
    df.equipment=df.equipment.str.strip().str.upper()
  
    df.drop_duplicates(['airline','source_airport','destination_airport'],inplace=True)
    df.reset_index(drop=True,inplace=True)
    return df

In [142]:
def create_database_schema(cur,conn):
        cur.execute("CREATE EXTENSION IF NOT EXISTS postgis;")

        cur.execute("DROP TABLE IF EXISTS routes CASCADE;")
        cur.execute("DROP TABLE IF EXISTS airports CASCADE;")
        cur.execute("DROP TABLE IF EXISTS airlines CASCADE;")
         # Create airports table
        cur.execute("""
            CREATE TABLE airports (
            airport_id INT PRIMARY KEY,
            name VARCHAR(255) NOT NULL,
            city VARCHAR(255),
            country VARCHAR(255),
            iata_code VARCHAR(5) ,
            icao_code VARCHAR(5) ,
            latitude DOUBLE PRECISION,
            longitude DOUBLE PRECISION,
            tltitude DOUBLE PRECISION,
            altitude INT,
            timezone NUMERIC,
            dst CHAR(1),
            tz_database VARCHAR(50),
            type VARCHAR(20),
            source VARCHAR(50)
            
        );

       
        """)
        
        # Create airlines table
        cur.execute("""
            CREATE TABLE airlines (
            airline_id INT PRIMARY KEY,
            name VARCHAR(255) NOT NULL,
            alias VARCHAR(255),
            iata_code VARCHAR(5) ,
            icao_code VARCHAR(5) ,
            callsign VARCHAR(255),
            country VARCHAR(255),
            active CHAR(1)
                );
        """)
        
        # Create routes table
        cur.execute("""
            CREATE TABLE routes (
            route_id SERIAL PRIMARY KEY,
            airline VARCHAR(50),
            airline_id INT NOT NULL,
            source_airport VARCHAR(50),	
            source_airport_id INT NOT NULL,
            destination_airport VARCHAR(50),
            destination_airport_id INT NOT NULL,
            codeshare CHAR(1),
            stops INT DEFAULT 0,
            equipment VARCHAR(255),

            -- Foreign key to airlines
            FOREIGN KEY (airline_id) REFERENCES airlines(airline_id) ON DElETE CASCADE,

            -- Foreign key to airports (source airport)
            FOREIGN KEY (source_airport_id) REFERENCES airports(airport_id) ON DElETE CASCADE,

            -- Foreign key to airports (destination airport)
            FOREIGN KEY (destination_airport_id) REFERENCES airports(airport_id) ON DElETE CASCADE
        );

        """)
        
        cur.execute("""
                    
        SELECT AddGeometryColumn('airports', 'location', 4326, 'POINT', 2);

        -- Update the coordinates
        UPDATE airports 
        SET location = ST_SetSRID(ST_MakePoint(longitude, latitude), 4326);

            -- Indexes on foreign keys in the routes table
                    
            CREATE INDEX idx_routes_airline_id ON routes (airline_id);
            CREATE INDEX idx_routes_source_airport_id ON routes (source_airport_id);
            CREATE INDEX idx_routes_destination_airport_id ON routes (destination_airport_id);

            -- Geospatial index on the location column in airports table for fast geospatial queries
                    
            CREATE INDEX idx_airports_location ON airports USING GIST (location);
        """)

## 1. Data Ingestion:


In [143]:
engine = create_engine(
            f'postgresql://{user}:{password}@'
            f'{host}:{port}/{database}'
        )
conn = psycopg2.connect(host=f"{host}",database=f"{database}",user=f'{user}',password=f'{password}',port=f'{port}')
conn.set_session(autocommit=True)
cur = conn.cursor()   
airports = download_and_clean_airport_df()
airlines = download_and_clean_airline_df()
routes = download_and_clean_routes_df()


7698
6162
67663


In [144]:
create_database_schema(cur=cur,conn=conn)

## Uploaed To Database

In [145]:
# upload airports df to postgres database as  airports table
# Define the dtype mapping for the columns
dtypes = {
        'airport_id': Integer,
        'name': String,
        'city': String,
        'country': String,
        'iata_code': String,
        'icao_code': String,
        'latitude': Float,
        'longitude': Float,
        'altitude': Integer,
        'timezone': Float,
        'dst': String,
        'tz_database': String,
        'type':String,
        'source':String,
        'location': Geometry('POINT', srid=4326)
    }

    # upload into database
airports.to_sql(
        'airports',
        engine,
        if_exists='append',
        index=False,
        dtype=dtypes
    )

345

In [146]:
# upload airlines df to postgres database as  airlines table
# Define the dtype mapping for the columns
dtpes_airlines = {
    
        'airline_id': Integer,
        'name': String,
        'alias': String,
        'iata_code': String,
        'icao_code': String,
        'callsign': String,
        'country': String,
        'active': String
}
airlines.to_sql('airlines', engine, if_exists='append', index=False) 

532

In [147]:
# upload routes df to postgres database as  routes table
# Define the dtype mapping for the columns
dtypes_routes = {
        'airline': String,
        'airline_id': Integer,
        'source_airport': String,
        'source_airport_id': Integer,
        'destination_airport': String,
        'destination_airport_id': Integer,
        'codeshare': String,
        'stops': Integer,
        'equipment': String,
        
    }
routes.to_sql('routes', engine, if_exists='replace',dtype=dtypes_routes, index=False,)      

765

##  Data Enrichment:

In [148]:
def distance_between_airports_sql():
    """1. Calculate Distance Between Airports for Direct Flights"""
    
    cur.execute("""
                ALTER TABLE routes ADD COLUMN distance_km DOUBLE PRECISION;

                UPDATE routes
                SET distance_km = ST_DistanceSphere(
                    (SELECT location FROM airports WHERE airport_id = source_airport_id),
                    (SELECT location FROM airports WHERE airport_id = destination_airport_id)
                ) / 1000
                where stops=0
                    """)
distance_between_airports_sql()

## Data Querying

## Find the Top 3 Airlines that Serve the Most Routes in Each Country

In [149]:
pd.read_sql(""" select country, name as airline_name, route_count from (
							SELECT a.airline_id, a.name, a.country, COUNT(*) as route_count,
									ROW_NUMBER() OVER (PARTITION BY a.country ORDER BY COUNT(*) DESC) as rank
							FROM airlines a
									JOIN routes r ON a.airline_id = r.airline_id
									GROUP BY a.airline_id, a.name, a.country
						) as new_table
						WHERE rank <= 3 
						ORDER BY country,route_count desc
				""",engine).head(10)

,country,airline_name,route_count
0,Afghanistan,Ariana Afghan Airlines,22
1,Afghanistan,Safi Airlines,14
2,Afghanistan,Kam Air,5
3,Alaska,Alaska Airlines,530
4,Albania,Albanian Airlines,4
5,Algeria,Air Algerie,226
6,American Samoa,Baikotovitchestrian Airlines,41
7,Angola,Taag Angola Airlines,71
8,Antigua And Barbuda,Leeward Islands Air Transport,62
9,Argentina,Aerolineas Argentinas,154


## Identify the Airports with the Most Codeshare Flights Along with the Number of Codeshare Routes

In [150]:
pd.read_sql("""SELECT a.name as airport_name, COUNT(*) as codeshare_routes
                    FROM airports a
                    JOIN routes r ON a.airport_id = r.source_airport_id
                    WHERE r.codeshare = 'Y'
                    GROUP BY a.airport_id, a.name
                    ORDER BY codeshare_routes DESC

             """,engine).head(10)

,airport_name,codeshare_routes
0,Hartsfield Jackson Atlanta International Airport,633
1,Chicago O'Hare International Airport,277
2,Los Angeles International Airport,209
3,London Heathrow Airport,202
4,Vienna International Airport,159
5,Charles De Gaulle International Airport,155
6,Frankfurt Am Main Airport,150
7,Amsterdam Airport Schiphol,145
8,Dallas Fort Worth International Airport,143
9,Denver International Airport,136


## Calculate the Distance Between Airports for Direct Flights (Zero Stops) and Find the Longest Flight

In [151]:
pd.read_sql("""SELECT 
                        al.name as airline_name,
                        src.name as source_airport,
                        dst.name as destination_airport,
                        r.distance_km
                    FROM routes r
                    JOIN airports src ON r.source_airport_id = src.airport_id
                    JOIN airports dst ON r.destination_airport_id = dst.airport_id
                    JOIN airlines al ON r.airline_id = al.airline_id
                    WHERE r.stops = 0
                    ORDER BY r.distance_km DESC
        """,engine).head(1)

,airline_name,source_airport,destination_airport,distance_km
0,Proflight Commuter Services,Kenneth Kaunda International Airport Lusaka,Los Alamitos Army Air Field,16082.277341


## Find the Airport Hosting the Most Unique Airlines Based on Routes Operated

In [152]:
pd.read_sql("""SELECT 
                a.name as airport_name, a.country,
                COUNT(DISTINCT r.airline_id) as unique_airlines
            FROM airports a
            JOIN routes r ON a.airport_id = r.source_airport_id
            GROUP BY a.airport_id, a.name,a.country
            ORDER BY unique_airlines DESC
            LIMIT 10;
            """,engine)    

,airport_name,country,unique_airlines
0,Charles De Gaulle International Airport,France,108
1,Frankfurt Am Main Airport,Germany,100
2,Suvarnabhumi Airport,Thailand,98
3,Leonardo Da Vinci–Fiumicino Airport,Italy,92
4,London Heathrow Airport,United Kingdom,86
5,Dubai International Airport,United Arab Emirates,83
6,Hong Kong International Airport,Hong Kong,83
7,Singapore Changi Airport,Singapore,83
8,Beijing Capital International Airport,China,81
9,Amsterdam Airport Schiphol,Netherlands,79


## Find the Airline Covering the Widest Range of Geographic Latitudes or Longitudes

In [153]:
pd.read_sql("""SELECT 
    al.name as airline_name,r.airline,
                MAX(src.latitude) - MIN(src.latitude) as latitude_range
            FROM routes r
            JOIN airports src ON r.source_airport_id = src.airport_id
            JOIN airports dst ON r.destination_airport_id = dst.airport_id
            JOIN airlines al ON r.airline_id = al.airline_id
            GROUP BY al.airline_id, al.name,r.airline
            ORDER BY latitude_range DESC
            """,engine).head(10)

,airline_name,airline,latitude_range
0,Klm Royal Dutch Airlines,KL,113.738102
1,Iberia Airlines,IB,113.319802
2,Finnair,AY,106.280602
3,Lan Airlines,LA,104.876633
4,Emirates,EK,104.821400
5,Qantas,QF,104.673000
6,American Airlines,AA,104.663799
7,Lufthansa,LH,104.505496
8,Tam Brazilian Airlines,JJ,104.473202
9,Ethiopian Airlines,ET,103.806599


## Identify Cities Serving Multiple Types of Transport

In [154]:
pd.read_sql("""SELECT city,type, COUNT(DISTINCT type) AS transport_types
                FROM airports
                GROUP BY city,type
               	having count(DISTINCT type)>=1
                """,engine).head(10)

,city,type,transport_types
0,108 Mile Ranch,airport,1
1,Aachen,airport,1
2,Aalborg,airport,1
3,Aalen-Heidenheim,airport,1
4,Aappilattoq,airport,1
5,Aarhus,airport,1
6,Aasiaat,airport,1
7,Abadan,airport,1
8,Abaiang Atoll,airport,1
9,Abakan,airport,1


## Airports with the Highest Number of Airlines Operating and Routes

In [155]:
pd.read_sql("""WITH airport_stats AS (
                        SELECT 
                            a.airport_id,
                            a.name,
                            COUNT(DISTINCT r.airline_id) as airline_count,
                            COUNT(*) as route_count
                        FROM airports a
                        JOIN routes r ON a.airport_id = r.source_airport_id
                        GROUP BY a.airport_id, a.name
                    )
                    SELECT 
                        name as airport_name,
                        airline_count,
                        route_count
                    FROM airport_stats
                    order by route_count desc
                """,engine).head(10)

,airport_name,airline_count,route_count
0,Hartsfield Jackson Atlanta International Airport,37,915
1,Chicago O'Hare International Airport,46,558
2,Beijing Capital International Airport,81,535
3,London Heathrow Airport,86,525
4,Charles De Gaulle International Airport,108,524
5,Frankfurt Am Main Airport,100,497
6,Los Angeles International Airport,62,492
7,Dallas Fort Worth International Airport,33,469
8,John F Kennedy International Airport,74,456
9,Amsterdam Airport Schiphol,79,453
